In [70]:
# Install
!pip install yfinance
!pip install crewai crewai-tools
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install -U duckduckgo-search

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [71]:
# Import
import json
import os
from datetime import datetime

import numpy as np
np.float_ = np.float64

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [72]:
# Create Yahoo Finance Tool
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return stock

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetches stock prices for {ticket} from the last year about a specific company from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [73]:
# Import OPENAI LLM - GPT
os.environ['OPENAI_API_KEY'] = "apikeyhere"
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [74]:
# Create stock analyst agent
stockPriceAnalyst = Agent(
  role = "Senior stock price Analyst",
  goal = "Find the {ticket} stock price and analyses trends",
  backstory="""You're a highly experienced in analyzing the price of an specific stock
  and make predictions about its future price""",
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = False,
  tools = [yahoo_finance_tool]
)

In [75]:
# Create stock analysis task
getStockPrice = Task(
  description = "Analyze the stock {ticket} price history and create a trend analysis of up, down or sideways",
  expected_output = """ Specify the current trend stock price - up, down, or sideways.
  eg. stock = 'APPL, price UP'
  """,
  agent = stockPriceAnalyst
)

In [76]:
# Create news analysis search tool
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [77]:
# Create news analysis agent
newsAnalyst = Agent(
  role = "Stock news Analyst",
  goal = """Create a short summary of the market news related to the stock {ticket} company.
  Specify the current trend - up, down or sideways with the news context.
  For each requested stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.
  """,
  backstory="""You're a highly experienced in analyzing the market trends and news and have tracked assets for more than 10 years.
  You're also master level analyst in traditional markets and have deep understanding of human psychology.
  You understand news, their titles and information, but you look at those with a health dose of skepticism.
  You consider also the source of the news articles.
  """,
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  allow_delegation = False,
  tools = [search_tool]
)

In [78]:
# Create news analysis task
get_news = Task(
  description = f"""Take the stock and always include BTC to it (if not requested).
  Use the search tool to search each one individually.

  The current date is {datetime.now()}.
  
  Compose the results into a helpful report.
  """,
  expected_output = """A summary of the overall market and one sentence summary for each requested asset.
  Include a fear/greed score for each asset based on the news. Use the format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent = newsAnalyst
)

In [79]:
# Create stock report agent
stockAnalystWriter = Agent(
  role = "Senior Stock Analyst Writer",
  goal = """Analyze the trends price and news and write an insightful, compelling and informative 3 paragraph long newsletter based on the stock report and price trend.""",
  backstory="""You're widely accepted as the best stock analyst in the market.
  You understand complex concepts and create compelling stories and narratives that resonate with wider audiences.
  You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analysis.
  You're able to hold multiple opinions when analyzing anything.
  """,
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  allow_delegation = True,
)

In [80]:
# Create stock report task
writeAnalysis = Task(
  description = """Use the stock price trend and the stock news report to create an analysis and write the newsletter about the {ticket} company
  that is brief and highlights the most important points.
  Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
  Include the previous analysis of the stock trend and news summary 
  """,
  expected_output = """An eloquent 3 paragraph newsletter formatted as markdown in an easy, readable manner.
  It should contain:
  - 3 bullets executive summary.
  - Introduction - set the overall picture and spike up the interest.
  - Main part provides the meat of the analysis including the news summary and fear/greed scores.
  - Summary - key facts and concrete future trend prediction - up, down or sideways.
  """,
  agent = stockAnalystWriter,
  context = [getStockPrice, get_news]
)

In [94]:
crew = Crew(
  agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWriter],
  tasks = [getStockPrice, get_news, writeAnalysis],
  verbose = True,
  process = Process.hierarchical,
  full_output = True,
  share_crew = False,
  manager_llm = llm,
  max_iter = 15
)


2024-08-20 22:22:04,491 - 139930180153344 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [95]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

 [2024-08-20 22:22:06][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 22:22:06][INFO]: == Starting Task: Analyze the stock AAPL price history and create a trend analysis of up, down or sideways


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-20 22:23:32][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 22:23:32][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 22:23:32][INFO]: == Starting Task: Take the stock and always include BTC to it (if not requested).
  Use the search tool to search each one individually.

  The current date is 2024-08-20 21:58:11.480661.
  
  Compose the results into a helpful report.
  


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-20 22:24:57][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 22:24:58][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 22:24:58][INFO]: == Starting Task: 

In [97]:
list(results.keys())

AttributeError: 'CrewOutput' object has no attribute 'keys'

In [98]:
results['final_output']

TypeError: 'CrewOutput' object is not subscriptable

In [ ]:
len(results['tasks_outputs'])

In [ ]:
Markdown(results['final_output'])